In [ ]:
%%writefile word_count_mapper.py
# in python 2

import sys, io
import re

reload(sys)
sys.setdefaultencoding('utf-8')

for line in sys.stdin:
    try:
        article_id, content = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    word_list = re.split("\W*\s+\W*", content, flags=re.UNICODE)
    for word in word_list:
        print >> sys.stderr, "reporter:counter:word_tracker,total_word,%d" %1
        print "%s\t%d" %(word.lower(), 1)

In [ ]:
%%writefile word_count_reducer.py
# in python 2

import sys
current_word = None
word_counts = 0

for line in sys.stdin:
    try:
        word, counts = line.strip().split("\t",1)
        counts = int(counts)
    except ValueError as e:
        continue
    if word != current_word:
        if current_word:
            print "%s\t%d" %(current_word, word_counts)
        current_word = word
        word_counts = 0
    word_counts += counts
    
if current_word:
    print "%s\t%d" %(current_word, word_counts)

In [ ]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files word_count_mapper.py,word_count_reducer.py \
    -mapper "python word_count_mapper.py" \
    -combiner "python word_count_reducer.py" \
    -reducer "python word_count_reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head